# **Convolutional Neural Networks**

In the previous section, we built and trained a simple model to classify ASL images. The model was able to learn how to correctly classify the training dataset with very high accuracy, but, it did not perform nearly as well on validation dataset. This behavior of not generalizing well to non-training data is called [overfitting](https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html), and in this section, we will introduce a popular kind of model called a [convolutional neural network](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53) that is especially **good for reading images and classifying them.**

## Objectives

* Prepare data specifically for a CNN
* Create a more sophisticated CNN model, understanding a greater variety of model layers
* Train a CNN model and observe its performance

## Loading and Preparing the Data

In [1]:
import tensorflow.keras as keras
import pandas as pd

# load in our data from CSV files
train_df = pd.read_csv("/content/sign_mnist_train.csv")
valid_df = pd.read_csv("/content/sign_mnist_valid.csv")

# separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

## Reshaping Images for a CNN

The individual pictures in our dataset are in the format of long lists of 784 pixels:

In [2]:
x_train.shape, x_valid.shape

((27455, 784), (7172, 784))

In this format, we don't have all the information about which pixels are near each other. Because of this, we can't apply convolutions that will detect features. Let's **reshape our dataset so that they are in a 28x28 pixel format. This will allow our convolutions to associate groups of pixels and detect important features.**

Note that for the first convolutional layer of our model, we need to have not only the height and width of the image, but also the number of [color channels](https://www.photoshopessentials.com/essentials/rgb/). Our images are **grayscale**, so we'll just have **1 channel.**

That means that we need to **convert the current shape** `(27455, 784)` to `(27455, 28, 28, 1)`. As a convenience, we can pass the [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy.reshape) method a `-1` for any dimension we wish to remain the same, therefore:

In [3]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [4]:
x_train.shape

(27455, 28, 28, 1)

In [5]:
x_valid.shape

(7172, 28, 28, 1)

In [6]:
x_train.shape, x_valid.shape

((27455, 28, 28, 1), (7172, 28, 28, 1))

## Creating a Convolutional Model

These days, many data scientists start their projects by borrowing model properties from a similar project. Assuming the problem is not totally unique, there's a great chance that people have created models that will perform well which are posted in online repositories like [TensorFlow Hub](https://www.tensorflow.org/hub) and the [NGC Catalog](https://ngc.nvidia.com/catalog/models). Today, we'll provide a model that will work well for this problem.

<center><img src="https://i.postimg.cc/xdhhXptj/cnn.png" width="200"></center>

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))

### [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

<center><img src="https://i.postimg.cc/j2mLmbX0/conv2d.png" width="400"></center>

These are our 2D convolutional layers. **Small kernels will go over the input image and detect features that are important for classification.** Earlier convolutions in the model will detect simple features such as lines. Later convolutions will detect more complex features. Let's look at our first Conv2D layer:
```Python
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same'...)
```
75 refers to the **number of filters that will be learned.** (3,3) refers to the **size of those filters.** Strides refer to the **step size that the filter will take as it passes over the image**. Padding refers to **whether the output image that's created from the filter will match the size of the input image.** 

### [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization)

Like normalizing our inputs, **batch normalization scales the values in the hidden layers to improve training.** [Read more about it in detail here](https://towardsdatascience.com/batch-normalization-in-neural-networks-1ac91516821c). 

### [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)

<center><img src="https://i.postimg.cc/vm08CyYq/maxpool2d.png" width="400"></center>

**Max pooling takes an image and essentially shrinks it to a lower resolution.** It does this to help the model be robust to translation (objects moving side to side), and also makes our model faster.

### [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)

<center><img src="https://i.postimg.cc/X7gv99M6/dropout.png" width="400"></center>

**Dropout is a technique for preventing overfitting.** Dropout randomly selects a subset of neurons and turns them off, so that they do not participate in forward or backward propagation in that particular pass. This helps to make sure that the network is robust and redundant, and does not rely on any one area to come up with answers.    

### [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)

**Flatten takes the output of one layer which is multidimensional, and flattens it into a one-dimensional array.** The output is called a **feature vector** and will be connected to the final classification layer.

### [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)

Our first dense layer (512 units) **takes the feature vector as input** and learns which features will contribute to a particular classification. The second dense layer (24 units) is the **final classification layer** that outputs our prediction.

## Summarizing the Model

Here, we summarize the model we just created. **Notice how it has fewer trainable parameters** than the model in the previous notebook:

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 75)        750       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 75)       300       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 75)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        33800     
                                                                 
 dropout (Dropout)           (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 50)       2

## Compiling the Model

We'll compile the model just like before:

In [9]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

## Training the Model

**Despite the very different model architecture, the training looks exactly the same.** The cell below will train for 20 epochs and let's see if the accuracy improves:

In [10]:
model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_valid, y_valid))

Epoch 1/20
858/858 [==============================] - 71s 81ms/step - loss: 0.3032 - accuracy: 0.9085 - val_loss: 0.3312 - val_accuracy: 0.8928
Epoch 2/20
858/858 [==============================] - 65s 75ms/step - loss: 0.0167 - accuracy: 0.9944 - val_loss: 0.1648 - val_accuracy: 0.9600
Epoch 3/20
858/858 [==============================] - 64s 75ms/step - loss: 0.0093 - accuracy: 0.9973 - val_loss: 0.6309 - val_accuracy: 0.8868
Epoch 4/20
858/858 [==============================] - 64s 74ms/step - loss: 0.0058 - accuracy: 0.9982 - val_loss: 0.2201 - val_accuracy: 0.9488
Epoch 5/20
858/858 [==============================] - 64s 74ms/step - loss: 0.0042 - accuracy: 0.9988 - val_loss: 1.2172 - val_accuracy: 0.8093
Epoch 6/20
858/858 [==============================] - 64s 74ms/step - loss: 0.0051 - accuracy: 0.9987 - val_loss: 0.2997 - val_accuracy: 0.9541
Epoch 7/20
858/858 [==============================] - 64s 75ms/step - loss: 0.0059 - accuracy: 0.9987 - val_loss: 0.2704 - val_accuracy:

## Discussion of Results

It looks like this model has significantly improved! **The training accuracy is very high, and the validation accuracy has improved as well.** This is a great result, as all we had to do was swap in a new model.

You may have noticed **the validation accuracy jumping around.** This is an indication that our **model is still not generalizing perfectly.** 

## Summary

In this section, we utilized several new kinds of layers to implement a CNN, which performed better than the more simple model used in the last section. Hopefully the overall process of creating and training a model with prepared data is starting to become even more familiar.

<hr>

**Source**: [NVIDIA Deep Learning Institute](https://www.nvidia.com/en-us/training/)

<hr>